In [21]:
#import important stuff
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.spectrum.band_structure import BandStructure
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt
import ase.units
Hartree = ase.units.Hartree#Hartree energy
Bohr = ase.units.Bohr#Bohr radius
#
formula = 'MoS2'
sym_path='KG'
nP=30
emptybands=13
bs1=BandStructure.read('formula_'+formula+'_direction_'+sym_path+'_npoints_{}_emptybands_{}'.format(nP,emptybands)+'.json')
#
Nkpt=np.shape(bs1.energies)[1]
kPath=bs1.path.kpts
kDiff=kPath-kPath[0]
DeltaK_mag=np.zeros((Nkpt))
for i in range(Nkpt):
    DeltaK_mag[i]=np.sqrt((kPath[i,0]-kPath[0,0])**2+(kPath[i,1]-kPath[0,1])**2+(kPath[i,2]-kPath[0,2])**2)
#
energybands=bs1.energies[0]
valenceBand=energybands[:,12]
conductionBand=energybands[:,13]
#choose a cut off
k_cut=0.03
bool_k=DeltaK_mag<k_cut
DeltaK_cut=DeltaK_mag[bool_k]
valenceBand_cut=valenceBand[bool_k]
conductionBand_cut=conductionBand[bool_k]
#
kVec=np.concatenate((np.sort(-DeltaK_cut),DeltaK_cut[1:]))
valVec=np.concatenate((np.sort(valenceBand_cut),valenceBand_cut[1:]))
conVec=np.concatenate((-np.sort(-conductionBand_cut),conductionBand_cut[1:]))
print(kVec)
print(valVec)
print(conVec)
#
#fit
from scipy.optimize import curve_fit
xData=kVec
y1Data=valVec
y2Data=conVec
#fit function
atomUnit_to_eV=Hartree
Aang_to_atomUnit=1/Bohr
#apparently k is already in atomic units!!!
def val_band(x,A1,m_eff_h):
    return A1-atomUnit_to_eV*(x)**2/(2*m_eff_h)
def con_band(x,A1,m_eff_e):
    return A1+atomUnit_to_eV*(x)**2/(2*m_eff_e)
parameters1, _ = curve_fit(val_band, xData, y1Data)
parameters2, _ = curve_fit(con_band, xData, y2Data)
print(parameters1)
print(parameters2)
fit_val=val_band(xData,parameters1[0],parameters1[1])
fit_con=con_band(xData,parameters2[0],parameters2[1])
print(fit_val)
#print(valVec)
print(fit_con)

[-0.02570193 -0.          0.02570193]
[-4.07504181 -4.06083408 -4.07504181]
[-2.38184421 -2.39920275 -2.38184421]
[-4.06083408  0.63259741]
[-2.39920275  0.51777258]
[-4.07504181 -4.06083408 -4.07504181]
[-2.38184421 -2.39920275 -2.38184421]


In [16]:
plt.plot(kVec,valVec,'r-*')
plt.plot(kVec,fit_val,'b-*')
plt.legend(['Data','Fit'])
plt.xlabel('Delta k')
plt.ylabel('E [eV]')
plt.title('Effective h-mass: {}'.format(parameters1[1]))
plt.savefig('./plots/valFit_{}_sym_{}_npoints_{}_emptybands{}.svg'.format(formula,sym_path,nP,emptybands))
plt.close()

plt.plot(kVec,conVec,'r-*')
plt.plot(kVec,fit_con,'b-*')
plt.legend(['Data','Fit'])
plt.xlabel('Delta k')
plt.ylabel('E [eV]')
plt.title('Effective e-mass: {}'.format(parameters2[1]))
plt.savefig('./plots/conFit_{}_sym_{}_npoints_{}_emptybands_{}.svg'.format(formula,sym_path,nP,emptybands))
plt.close()

In [22]:
#der er et problem for KM. Jeg ved ikke hvorfor, men det er en god anledning til at finde 2nd derivative og bruge den
DeltaK_quan=DeltaK_mag[1]
Dev2_val=2*(valenceBand[1]-valenceBand[0])/DeltaK_quan**2
Dev2_con=2*(conductionBand[1]-conductionBand[0])/DeltaK_quan**2
#valenceBand=energybands[:,12]
#conductionBand=energybands[:,13]
m_eff_e=atomUnit_to_eV/Dev2_con
m_eff_h=-atomUnit_to_eV/Dev2_val
print(m_eff_e,m_eff_h)
#it works!!

0.5177725776140991 0.6325974117525929
